<a href="https://colab.research.google.com/github/eashwar05/CalQuity-task/blob/main/adaptive_event_driven_portfolio_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:0

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from crewai import Agent
import requests
from textblob import TextBlob
from datetime import datetime, time, date
import pytz

In [ ]:
from google.colab import userdata
news = userdata.get('news_api')

In [ ]:
def fetch_event_data(tickers, period='1y'):
    data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        history = stock.history(period=period)
        actions = stock.actions  # dividends, splits, etc.
        calendar = stock.calendar  # earnings, etc.
        data[ticker] = {"history": history, "actions": actions, "calendar": calendar}
    return data

In [ ]:
def fetch_news_headlines(ticker):
    # Use your actual API key in place of 'YOUR_API_KEY'
    url = f"https://newsapi.org/v2/everything?q={ticker}&apiKey=53bf5843cc034cda9cdbcb7fe3494ab8"
    response = requests.get(url)
    if response.ok:
        articles = response.json().get("articles", [])
        headlines = [article["title"] for article in articles]
        return headlines
    else:
        # Print or log the error message for debugging
        print("Error fetching news:", response.json())
        return []  # Return an empty list if the API call failed

# Test the function
#print(fetch_news_headlines('AAPL'))


In [ ]:
class DataCollectorAgent(Agent):
    role: str = "Data Collector"
    goal: str = "Fetch historical stock data, corporate actions, and news headlines."
    backstory: str = "This agent is responsible for gathering financial data from external APIs."

    def run(self, tickers):
        print(f"[DataCollectorAgent] Fetching data for: {tickers}")
        data = fetch_event_data(tickers)
        for ticker in tickers:
            data[ticker]['news'] = fetch_news_headlines(ticker)
        print("[DataCollectorAgent] Data collection complete.")
        return data

In [ ]:
def calculate_risk_metrics(prices):
    returns = prices.pct_change().dropna()
    volatility = returns.std()
    sharpe_ratio = returns.mean() / volatility if volatility != 0 else 0
    drawdown = (prices / prices.cummax() - 1).min()
    return {"volatility": volatility, "sharpe_ratio": sharpe_ratio, "drawdown": drawdown}

In [ ]:
def risk_analysis_around_event(prices, event_date, window=30):
    # If event_date is a date but not a datetime, convert it to datetime.
    if isinstance(event_date, date) and not isinstance(event_date, datetime):
        # Combine the date with the minimum time to get a datetime object.
        event_datetime = datetime.combine(event_date, time.min)
    else:
        event_datetime = event_date

    # Ensure the prices index is timezone-naive for comparison
    if prices.index.tz is not None:
        prices.index = prices.index.tz_localize(None)

    # Find the closest index to event_datetime using get_indexer (method 'nearest' allowed via get_indexer)
    nearest_index = prices.index.get_indexer([event_datetime], method='nearest')[0]
    event_datetime_aligned = prices.index[nearest_index]

    pre_event = prices.loc[:event_datetime_aligned].tail(window)
    post_event = prices.loc[event_datetime_aligned:].head(window)

    return {
        "pre_event": calculate_risk_metrics(pre_event),
        "post_event": calculate_risk_metrics(post_event)
    }

# Example usage in risk analysis error logging:
def analyze_events_for_ticker(ticker, event_dates, prices):
    results = {}
    for event_date in event_dates:
        try:
            results[str(event_date)] = risk_analysis_around_event(prices, event_date)
        except Exception as e:
            # Instead of calling .date(), simply use str() to print the event_date.
            print(f"Error analyzing event for {ticker} on {str(event_date)}: {e}")
    return results


In [ ]:
class RiskAnalyzerAgent(Agent):
    role: str = "Risk Analyzer"
    goal: str = "Compute risk metrics and event impact on asset prices."
    backstory: str = "This agent analyzes stock data to identify changes in risk around financial events."

    def run(self, stock_data):
        results = {}
        for ticker, data in stock_data.items():
            history = data["history"]
            prices = history['Close']
            overall_metrics = calculate_risk_metrics(prices)
            event_analysis = {}
            # Attempt to analyze risk around each earnings event if available.
            earnings_dates = data.get("calendar", {}).get("Earnings Date", [])
            for event_date in earnings_dates:
                try:
                    event_analysis[str(event_date.date())] = risk_analysis_around_event(prices, event_date)
                except Exception as e:
                    print(f"Error analyzing event for {ticker} on {event_date}: {e}")
            results[ticker] = {"overall": overall_metrics, "events": event_analysis}
        print("[RiskAnalyzerAgent] Risk metrics calculated.")
        return results

In [ ]:
def analyze_sentiment(headlines):
    if headlines is None or not headlines:
        return 0
    sentiments = [TextBlob(headline).sentiment.polarity for headline in headlines]
    return sum(sentiments) / len(sentiments) if sentiments else 0


In [ ]:
def attach_sentiment_scores(stock_data):
    for ticker, data in stock_data.items():
        headlines = data.get("news", [])
        data["sentiment"] = analyze_sentiment(headlines)
    return stock_data


In [ ]:
def create_interactive_dashboard(prices, events, sentiment, ticker):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=prices.index, y=prices,
                             mode='lines', name=f'{ticker} Price'))
    # Loop over event dates and add vlines and annotations separately.
    for event_date in list(events):
        # Convert event_date to datetime if it isn't already.
        event_date_converted = pd.to_datetime(event_date)

        # Add a vertical line (without annotation_text).
        fig.add_vline(
            x=event_date_converted,
            line_width=2,
            line_dash="dash",
            line_color="red"
        )

        # Add a separate annotation for the event.
        fig.add_annotation(
            x=event_date_converted,
            y=prices.max(),   # Position annotation at top of chart (adjust as needed)
            text=f"Event:\n{event_date_converted.strftime('%Y-%m-%d')}",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40
        )

    # Add an annotation for the average sentiment.
    fig.add_annotation(
           x=prices.index[-1],
           y=prices.iloc[-1],
           text=f"Sentiment: {sentiment:.2f}",
           showarrow=False,
           font=dict(color="blue", size=12)
    )
    fig.update_layout(title=f"{ticker} Dashboard",
                      xaxis_title="Date", yaxis_title="Price")
    fig.show()


In [ ]:
class NewsSentimentAgent(Agent):
    role: str = "News Sentiment Agent"
    goal: str = "Analyze sentiment from news headlines to gauge market perception."
    backstory: str = "Processes financial news to provide a sentiment score for each asset."

    def run(self, stock_data):
        print("[NewsSentimentAgent] Analyzing news sentiment.")
        return attach_sentiment_scores(stock_data)

class VisualizerAgent(Agent):
    role: str = "Visualizer"
    goal: str = "Generate interactive dashboards for visualizing risk and sentiment analytics."
    backstory: str = "This agent creates visual representations of the analysis results."

    def run(self, stock_data):
        print("[VisualizerAgent] Generating visualizations.")
        for ticker, data in stock_data.items():
            prices = data["history"]['Close']
            events = data.get("actions", {}).index  # Use corporate actions as event markers example
            sentiment = data.get("sentiment", 0)
            create_interactive_dashboard(prices, events, sentiment, ticker)

In [ ]:
if __name__ == "__main__":
    tickers = ['AAPL', 'META']  # Define your target tickers
    # Instantiate agents
    data_agent = DataCollectorAgent()
    risk_agent = RiskAnalyzerAgent()
    sentiment_agent = NewsSentimentAgent()
    visualizer_agent = VisualizerAgent()

    # Step 1: Data Collection
    stock_data = data_agent.run(tickers)

    # Step 2: Risk Analysis
    risk_metrics = risk_agent.run(stock_data)

    # Step 3: News Sentiment Analysis
    stock_data = sentiment_agent.run(stock_data)

    # Step 4: Visualization
    visualizer_agent.run(stock_data)

    print("Adaptive Event-Driven Portfolio Optimizer pipeline complete.")


[DataCollectorAgent] Fetching data for: ['AAPL', 'META']
[DataCollectorAgent] Data collection complete.
Error analyzing event for AAPL on 2025-04-30: 'datetime.date' object has no attribute 'date'
Error analyzing event for AAPL on 2025-05-05: 'datetime.date' object has no attribute 'date'
Error analyzing event for META on 2025-04-22: 'datetime.date' object has no attribute 'date'
Error analyzing event for META on 2025-04-28: 'datetime.date' object has no attribute 'date'
[RiskAnalyzerAgent] Risk metrics calculated.
[NewsSentimentAgent] Analyzing news sentiment.
[VisualizerAgent] Generating visualizations.


Adaptive Event-Driven Portfolio Optimizer pipeline complete.
